# Data Management for Individual Scientists

Authors: Erik Tollerud & Brigitta Sipőcz

In Astronomy, "Data Management" is typically used to describe large-scale efforts like the Gigabytes-per-second Large Synoptic Survey Telescope or the over a hundred different observing modes James Webb Space Telescope.  But for an individual scientist, the general concept data management still applies, just in a very different sense: managing data from or for your own scientific projects.  This tutorial aims to suggest some guidelines and pitfalls for personal data management.

While this tutorial covers several levels of complexity, there is one golden rule, which you should remember even if you remember nothing else: Do Not Make Your Own Format. You need only examine the examples that are shown in [the astropy table reader docs](http://docs.astropy.org/en/stable/io/ascii/#supported-formats) or the related [fixed width gallery](http://docs.astropy.org/en/stable/io/ascii/fixed_width_gallery.html#fixed-width-gallery) to see the needless complexity that has been introduced by well-meaning astronomers that chose to roll their own.  While the tools available to us often make it easy, do your best to resist. Future collaborators, future co-workers, future students, and future you will thank you.

Note that while this tutorial is primarily based on Python and some parts are Python-specific to provide concrete examples, most of the guidelines discussed here apply to a range of approaches and languages.

# DM the easy way: "automatic" tools

## Integrated Caching

Some software packages provide caching - i.e., after a file is downloaded, it's automatically saved somewhere and used automatically the next time you ask for it. This can be a convenient way to ensure data you need for your work is available and limit your impact on remote servers that might provide the data.

As an example, consider the following function for downloading a Hubble Space Telescope image of one of the greatest galaxies - the Local Group dwarf GR8:

In [ ]:
from astropy.utils import data
from astropy.io import fits

gr8_url = 'https://archive.stsci.edu/pub/hlsp/angst/acs/hlsp_angst_hst_acs-wfc_10915-gr8_f814w_v1_ref.fits'
gr8_fn = data.download_file(gr8_url, cache=True)
fits.open(gr8_fn).info()

The first time you run this it should take a little while to run the download (it's a 70MB file), but if you run it again, you'll see it's almost instantaneous.  This is because the file has been saved in a (relatively hidden to you) location and is re-used when you ask for it again.

This may seem like an easy way to manage your data, but consider these cases:
* What happens if the remote file gets udated?
* What happens if you start running out of space and want to delete some of your old data files?

You can address these manually by running the cell below, but consider what happens if you lose this notebook sometime between now and when the problems above arise. For this reason, this general problem has been enshrined in an computer science/software engineering adage: https://martinfowler.com/bliki/TwoHardThings.html .

In [ ]:
data.clear_download_cache(gr8_url)  # you might not want to actually run this cell, as this file is re-used later

To see this problem a little more clearly, consider the code below:

In [ ]:
from astropy.coordinates import SkyCoord, EarthLocation

sc = SkyCoord(ra=1, dec=2, unit='deg',
              frame='fk5', obstime='2019-8-1',
              location=EarthLocation.of_site('kitt peak'))
sc.transform_to('altaz')

Depending on when/if you've last used code like this, it might take a little while to run the first time, because behind the scenes the code has to look up the exact orientation of the Earth on the day in question (something that is not fully predictible due to things like earthquakes and therefore requires data downloads).  These data change regularly, so behind the scenes careful management is required by the software to ensure the file stays up to date and you aren't constantly served an out-of-date file that gives inaccurate position information, ruining your precision science.

To sum up - while caching is a viable solution if the software you were using is careful about managing it for you, in general you should not rely on it unless you are sure the data are never going to change, are publicly available, and are small enough you don't have to worry about deleting them.

## Pickling

Now lets consider the topic of *saving* data (as opposed to getting it). Python and its wider ecosystem provides a few ways of doing this that are built-in and relatively easy.  But as with caching, the easiest ways come with certain pitfalls.

Consider the following generated image - how would we save it?

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

image_scaling_factor = 1e4
xpix, ypix = np.mgrid[:512, :512]
img = xpix * ypix + np.random.randn(512, 512) * image_scaling_factor
plt.imshow(img);

Python provides a built-in way to handle basic data like this, called "pickling":

In [ ]:
import pickle

data_to_save = {'image': img, 'xy': (xpix, ypix), 'image_scaling_factor': image_scaling_factor}
with open('mydata.pickle', 'wb') as f:
    pickle.dump(data_to_save, f)

In [ ]:
import os

os.listdir()

In [ ]:
with open('mydata.pickle', 'rb') as f:
    loaded_data = pickle.load(f)
loaded_data

This shows that you can save out the image and some extra information with a minimum of fuss, and load it again almost as simply. However, there are some serious drawbacks here - to start with, take a look at the size of the file:

In [ ]:
os.path.getsize('mydata.pickle')/1024/1024 # MB

How large do you expect? Why might this not be ideal, particularly if this were a significantly larger dataset?

Moreover, `pickle` has some more subtle drawbacks. The following cells illustrate some of these.

#### Pickle Issue 1

In [ ]:
with open('data_management.ipynb', 'r') as nb_file:
    data_to_save = {'image': img, 'file_to_open': nb_file}
    with open('mydata.pickle', 'wb') as f:
        pickle.dump(data_to_save, f)

This problem is straightforward: not all types are picklable at all.

#### Pickle Issue 2

Let's say you've decided to use Python for it's object-oriented power.  You decide to make an image generator *class* instead of pickling the data itself:

In [ ]:
class ImageGenerator:
    def __init__(self, imgsize, imgscl):
        self.imgsize = imgsize
        self.imgscl = imgscl
        
    def make_image(self):
        xpix, ypix = np.mgrid[:self.imgsize[0], :self.imgsize[1]]
        return xpix * ypix + np.random.randn(*xpix.shape) * self.imgscl

In [ ]:
imagegen = ImageGenerator((512, 512), 1e4)

with open('mydata.pickle', 'wb') as f:
    pickle.dump(imagegen, f)
with open('mydata.pickle', 'rb') as f:
    imagegen_loaded = pickle.load(f)
    
plt.imshow(imagegen_loaded.make_image());

So far so good.  But now let's say you realize you want to re-work the class to use a more useful variable name:

In [ ]:
class ImageGenerator:
    def __init__(self, image_size, image_scaling_factor):
        self.image_size = image_size
        self.image_scaling_factor = image_scaling_factor
        
    def make_image(self):
        xpix, ypix = np.mgrid[:self.image_size[0], :self.image_size[1]]
        return xpix * ypix + np.random.randn(*xpix.shape) * self.image_scaling_factor

with open('mydata.pickle', 'rb') as f:
    imagegen_loaded = pickle.load(f)
    
plt.imshow(imagegen_loaded.make_image())

As you can see, the old object unpickled just fine... but doesn't work!  This is a simple example of a more general problem that if you ever need to re-name, re-design or otherwise change something you've pickled, and often renders all your pickles somewhere between mildly broken and completely unpickleable. And that's bad.

#### Pickle Issue 3

Now lets say you get an email "from" a trusted collaborator who includes a file for you to use.  You good naturedly load it up and see this:

In [ ]:
this_is_a_very_safe_pickle_trust_me = b"\x80\x03cbuiltins\nexec\nq\x00X\x8b\x01\x00\x00\nimport base64\nexec(base64.b64decode(b'CnByaW50KCJJIENBTiBIQVogQ0hFRVNFQlVSR0VSLiBBbHNvIEkgaGFja2VkIHlvdXIgZGF0YS4iKQppZiAnaW1hZ2VnZW5fbG9hZGVkJyBpbiBnbG9iYWxzKCk6CiAgICBpbWFnZV90b19oYWNrID0gZ2xvYmFscygpWydpbWFnZWdlbl9sb2FkZWQnXQogICAgZm9yIGksIG5hbWUgaW4gZW51bWVyYXRlKGltYWdlX3RvX2hhY2suX19kaWN0X18pOgogICAgICAgIHNldGF0dHIoaW1hZ2VfdG9faGFjaywgbmFtZSwgJ/CfkIgnIGlmIGklMj09MCBlbHNlICfwn42UJykK'))\nq\x01\x85q\x02Rq\x03."

pickle.loads(this_is_a_very_safe_pickle_trust_me)
    
imagegen_loaded.imgsize, imagegen_loaded.imgscl

As this demonstrates, pickle is an inherently insafe format because it has the potential to execute any arbitrary code while being unpickled (to see how this was done, you can look at `safe.py` in this repo). That means you should never trust any pickle someone sends you... So it's effectively useless for sharing data with others. 

#### Pickle Issue 4

On top of all that, the pickle format *itself* changes over time, such that pickles produced by a newer Python may not work on older versions of Python, and essentially none of them work with any language other than Python.

Taken together, that means pickle, while very convenient, is not useful for anything beyond saving your *own* data if it's either very simple data, or something that you're sure won't ever change (and trust me... it will).

### Exercise: the numpy formats

Similar, but somewhat different from the pickle format are the `npy` and `npz` formats.  These are files that the `numpy` package can produce from `numpy` arrays.  While they provide a similar quick-and-easy way to save out data from Python, they also have their own drawbacks (some similar to Pickle, others less so).  Explore trying to replicate the pickle example but with the `numpy` formats, and compare and contrast the advantages/disadvantages.  Discuss with your neighbor if you are both willing and interested.

In [ ]:
np.save?

In [ ]:
np.savez?

In [ ]:
np.savez_compressed?

# DM the somewhat harder way: managing file formats

While the above shows some easy techniques for managing your own data, it also highlighted several drawbacks.  Here, let's consider the next stage in complexity: managing your data using standard (non language-specific) file formats. Because these approaches are not quite as flexible as tools like pickle, we split this discussion into two distinct data types: Tables/catalogs, and Images (which generalize to other structured numerical data objects like spectra or data cubes).

## Tables/catalogs

Much of astronomy (and other!) data are built around the concept of "catalogs" - i.e., tabular data where each row is an individual "object".  Lets start by creating an example catalog which we'll fill with random (but not totally implausible) numbers.  For this exercise we will use [astropy tables](https://docs.astropy.org/en/stable/table/) due to their large variety of astro-specific output formats, but the general approaches apply just as well to other Python tables (most notably Pandas data frames) or even tabular constructs native to languages other than Python.

In [ ]:
from astropy import table

n_rows = 100000

tab = table.Table()
tab['name'] = ['RNGC{:05}'.format(i) for i in range(n_rows)]
tab['ra'] = np.random.rand(n_rows)*360
tab['dec'] = np.random.rand(n_rows)*180 - 90
tab['ra'].unit = tab['dec'].unit = 'deg'
for band in 'ugriz':
    tab['mag_' + band] = np.random.rand(n_rows)*20
    tab['mag_' + band].unit = 'mag'
    tab['mag_' + band + '_unc'] = np.random.rand(n_rows)
    tab['mag_' + band + '_unc'].unit = 'mag'

tab.meta['description'] = 'The REALLY New General Catalog'

tab

#### CSV and related text Files

Let's try dumping to a plain-old Comma Separated Value (CSV) file:

In [ ]:
tab.write('tab.csv', format='ascii.csv')

#this shows the first few lines of the file
with open('tab.csv') as f:
    for i in range(3):
        print(f.readline(), end='')

This particular file is a faithful representation of the data. It also has the advantage of being easy-to-understand (a human can read it) and therefore portable to many different languages or contexts.

In [ ]:
print('CSV file size:', os.path.getsize('tab.csv')/1024/1024, 'MB')
print('In-memory table size:', sum([c.dtype.itemsize for c in tab.columns.values()])*len(tab)/1024/1024, 'MB')

However, as the above shows, it's not a particularly efficient representation.

### Aside Exercise

Can you work out theoretically how inefficient this representation is, at least for numbers like the above? (Assuming each digit is encoded in ASCII, meaning 8-bit per character).

Moreover, this is a lossy way of storing the table.  In particular it loses all the metadata.  For example:

In [ ]:
loaded_csv_tab = table.Table.read('tab.csv')

print(tab['ra'].unit, loaded_csv_tab['ra'].unit)
print(tab.meta, loaded_csv_tab.meta)

That is, the units of all of the columns, and the description of the catalog, have been lost in conversion to CSV.  That makes the file of dubious archival value.  In addition, it can make code that depends on knowledge of the units completely fail:

In [ ]:
SkyCoord.guess_from_table(tab)  # <- works because the ra/dec columns have units in them already

In [ ]:
SkyCoord.guess_from_table(loaded_csv_tab)  # <- the columns have no units and are therefore ambiguous

Fortunately there are other formats that can save these metadata while still maintaining some of the positive qualities of CSV files.  Most relevant of these is the ["ECSV" table format](https://github.com/astropy/astropy-APEs/blob/master/APE6.rst):

In [ ]:
tab.write('tab.ecsv', format='ascii.ecsv', delimiter=',')

with open('tab.ecsv') as f:
    for i in range(22):
        print(f.readline(), end='')

In [ ]:
loaded_ecsv_tab = table.Table.read('tab.ecsv')

print(tab['ra'].unit, loaded_ecsv_tab['ra'].unit)
print(tab.meta, loaded_ecsv_tab.meta)

As the above shows, this file is essentially a CSV file, but with a commented header to store the metadata. This addresses the round-tripping metadata problem, so ECSV files are are first really potentially sharable file: they can be self-descriptive if you create the table properly (as important for future you as for your current collaborators...), and are in a format that is potentially readable by a wide range of users, both Python-users and otherwise.

There are other astro-specific formats that store varying amounts of metadata (see the [list of supported writers](http://docs.astropy.org/en/stable/io/ascii/#supported-formats) for tables in the Astropy docs), of various popularity with archives, but ECSV is one of the simplest to use while still being complete, so we recommend it here.

### Binary tables

All text-based formats, however, suffer from inherent inefficiency (see the exercise above). To move beyond this we need to use more structured formats which allow binary tables.  A common example in astronomy is the FITS table format:

In [ ]:
tab.write('tab.fits', format='fits')

print('FITS file size:', os.path.getsize('tab.fits')/1024/1024, 'MB')
print('In-memory table size:', sum([c.dtype.itemsize for c in tab.columns.values()])*len(tab)/1024/1024, 'MB')

This immediately shows the value of binary tables: the size is notably smaller than for text tables.  The price paid is in interoperability: binary files in principle at least are not human-readable, and less interchangable because they can depend on the details of the system they are encoded on - while standards reduce this risk (i.e., the FITS standard mandates a specific byte order), it adds complexity to deal with this properly. While FITS is likely to remain a long-term stable standard, it is still a trade-off to consider for long-term archivability of research results.

However, we also see a subtle flaw here specific to FITS - the FITS standard is technically limited to a 8 character or less description.  In practice this is often ignored though, due to the FITS "HIERARCH" extension.  Does the reader then allow it to round-trip?:

In [ ]:
loaded_fits_tab = table.Table.read('tab.fits')

print(tab['ra'].unit, loaded_fits_tab['ra'].unit)
print(tab.meta, loaded_fits_tab.meta)

### VO Tables

One more table format is worth considering - the VO Table format.  VOTables are a standard adopted by many astronomical archives for storage and transfer of astronomical catalogs.  While VOTables are a more complex data structure that an individual table (they support multiple tables inside a single file, with shared columns and the like), in most cases it's straightforward to map the concepts. Moreover, the format supports both text *or* binary storage:

In [ ]:
from astropy.io import votable

vot = votable.from_table(tab)

vot.to_xml('tab.vot',  tabledata_format='tabledata')
vot.to_xml('tab.votb',  tabledata_format='binary2')

print('tabledata VOTable:', os.path.getsize('tab.vot')/1024/1024, 'MB')
print('binary VOTable:', os.path.getsize('tab.votb')/1024/1024, 'MB')

In binary form, this format is competitve with FITS and in someways more flexible, although as a result also quite a bit more complex (and hence).  The VO standards are also actively evolving, whereas FITS has been a stable format for many years.  In some ways this is better (long descriptions are legal) and some worse (the parser is slower do to the more complex format). In the end there's not a definitive answer - the choice is yours.

### Row vs Column-major tables

Before leaving the topic of catalogs, we should note one final consideration of relevance for Big Data problems: the layout of your catalogs.  The aforementioned foremats are all essentially row-major, in that they store catalogs as tables, where it's easiest to read a row at a time rather than a column at a time.  While the distinction is mostly academic for catalogs that fit in memory, for large data sets that might stress your computer's capabilities, storing data in a column-major form may be desirable.  As an example, a straightforward approach to this uses the capability of the hdf5 format to store columns as individual datasets to achive a column-major format.

In [ ]:
import h5py

hf = h5py.File('tab.hdf5', 'w')
hf.attrs['description'] = tab.meta['description']
for colname, col in tab.columns.items():
    if colname == 'name':
        col = col.astype('S9')
    ds = hf.create_dataset(colname, data=col)
    ds.attrs['unit'] = str(col.unit)
hf.close()

In [ ]:
hf = h5py.File('tab.hdf5', 'r')

hf['ra'][:] == tab['ra']
print(tab['ra'].unit, hf['ra'].attrs['unit'])

While not a drop-in replacement as some of the aforementioned formats are, this format can take advantage of some of HDF5's capabilities for parallelization and efficiency, and therefore stretch what your computer can do for catalogs that are near its limits.

## Images/data files

While the last part of this tutorial focused on catalogs, the other most significant forms of data to manage in astronomy are n-dimensional arrays.  The most common case is images, so we focus on those here, but the suggestions here apply just as well to spectra (or collections of spectra), data cubes, or even higher-dimensional constructs like time series if data cubes. 

### FITS

FITS is in its best form when used as a store of gridded data, as this is what the format was designed for. So unless you have a good reason not to (of which there certainly [are some!](https://doi.org/10.1016/j.ascom.2015.01.009)), it is still usually a good choice.  It particularly shines in its effective storage of World Coordinate Systems - for images this is typically the mapping from pixels coordinates to sky coordinates and vice versa.  For example:

In [ ]:
from astropy.wcs import WCS

# note that this will use the cached version if you did not clear it above 
gr8_fits = fits.open(gr8_url)

# turns out this file is slightly faulty and we have to fix it up...
for k in list(gr8_fits[0].header.keys()):
    if k.startswith('A_') or k.startswith('B_'):
        del gr8_fits[0].header[k]

w = WCS(gr8_fits[0].header)

In [ ]:
w.pixel_to_world([0], [1])

In [ ]:
gr8_coord = SkyCoord.from_name('GR 8')
w.world_to_pixel(gr8_coord)

Because the rules for FITS WCS are so standard, they have even been implemented into a matplotlib projection:

In [ ]:
plt.figure(figsize=(12, 15))
plt.subplot(projection=w)
plt.imshow(gr8_fits[0].data, vmin=-10, vmax=100);

While other formats can also represent WCS's in this manner, FITS' long history in astronomy ensures a wide range of tool compatibility, so it's frequently a good choice for images like this:

In [ ]:
gr8_fits[0].header['BUNIT'] = 'electron'
gr8_fits.writeto('gr8.fits', overwrite=True)

The above seems a bit silly when we *already* cached it, but you'll note it has been slightly modified from the original version.  Hence as an example of how you might work with a fits file you produced or modified, it is still sound.

### Exercise: file mangement

While the above describes working with individual FITS files, it does nothing to help you manage a large number of images like what might result from a full night's observing run. As an example of the sort of tool that can help with this problem, consider the [image management tools](https://ccdproc.readthedocs.io/en/latest/image_management.html) from the [ccdproc](https://ccdproc.readthedocs.io/) package.

If you have you are an observer, try using these tools on an observing run of your choice (follow ccdproc's [installation instructions](https://ccdproc.readthedocs.io/en/latest/install.html) if you do not have it already). Use the image magement tools to iterate over some sub-set of your observations and do some operation on them (which could be as simple as printing the header or `info()` of the file).

If you do not have your own image collections to use in this manner, ask a neighbor, consider downloading a few raw images from a public astronomy archive (perhaps via `astroquery` or from the [NOAO image archive](http://archive1.dm.noao.edu/search/query/)) and trying out ccdproc's tools on that, or just moving on if you're pressed for time.

### Language-level data structures

While up to this point we have discussed file formats, it is now a good time to ask the question: do we actually care *that* much about the file format? In reality, what we work with the most is the programming language interface to the data.  With this in mind, we should consider the value of language-level *data structures* for representing astronomical data like that we are discussing here. In particular, the use of the object-oriented paradigm allows us to focus on how we want to interact with the data instead of its format per se. An example of this is the `astropy.nddata` subpackage, which provides a range of classes for astronomical images.  Lets take a look at what this is like for our GR8 image:

In [ ]:
from astropy import nddata

gr8_ccd = nddata.CCDData.read('gr8.fits')

This `gr8_ccd` object now contains the file and its associated information, but has divorced it entirely from the context if the particular file format. It can now be worked with just as what it is: an image, rather than as a "fits image".  For example we can do cutouts using the embedded WCS information:

In [ ]:
from astropy import units as u

cutout = nddata.Cutout2D(gr8_ccd, gr8_coord, 15*u.arcsec, wcs=gr8_ccd.wcs)
plt.imshow(cutout.data, vmin=-10, vmax=100)

Or we can derive an uncertainty from the image, and then do  arithmetic operations that yield new CCDData objects that propogate uncertainties:

In [ ]:
# this number is just a guess extracted from the header to demonstrate the point, as the file doesn't contain a "real" uncertainty
gr8_ccd.uncertainty = nddata.VarianceUncertainty(77.0159555665 + gr8_ccd.data)

subgr8 = gr8_ccd.subtract(gr8_ccd)
subgr8

In [ ]:
plt.imshow(subgr8.uncertainty.array**0.5, vmin=3, vmax=20);

Despite the array itself being perfectly subtracted, the uncertainties were propogated as they should be in quadrature.

These `CCDData` objects can be written out in whatever format you desired, but that's mostly a detail - nothing in this concerned itself with the format except for the reading and writing. 

In [ ]:
subgr8.write('subgr8.fits')

This was meant as an example of this way of approaching data mangement (i.e., manging *data* not *files*). Other similar objects in use for astronomy include the [specutils](https://specutils.readthedocs.io/) package's [representations of astronomical spectra](https://specutils.readthedocs.io/en/stable/types_of_spectra.html), the [ndcube](https://docs.sunpy.org/projects/ndcube/en/stable/) package for storing data cubes (and higher-dimensionality cube analogs), as well as broader nd-array like constructs like [xarray](http://xarray.pydata.org/en/stable/) which are not astronomy-specific but are still valuable as starting points for astronomy applications.  

# "Industrial-grade" data management: databases

While not the focus of this tutorial, we will finish discussing how some of these concepts map to "industrial-scale" data management approaches that are generally built around databases. Because catalogs map best to the way database are typically laid out, we will use a simple embedded database that's built into Python and map our table onto that database:

(Note that you must have [pandas installed](https://pandas.pydata.org/pandas-docs/stable/install.html) for this to work)

In [ ]:
import sqlite3

# this special name means only keep the database in memory instead of on disk
sqldb = sqlite3.connect(':memory:')

# use pandas' SQLite writing machinery
tab.to_pandas().to_sql('astrotab', sqldb)

cursor = sqldb.execute('select * from astrotab')
cursor.description

Now this table operates just like a database - you can query it using standard SQL:

In [ ]:
cursor = sqldb.execute('select name,ra,dec, mag_r from astrotab WHERE mag_r>19')
cursor.fetchmany(5)

In this way, if you wished to, you can ingest your table into a database and start working on it as you would an external service otherwise too large to work with. This provides a gentle transition from individualized data management to industrial or mission-scale DM.

### Exercise

Try running an `astroquery` based ADQL or SQL query from a service that supports queries (e.g. SDSS or Gaia).  Try to get a reasonable-sized dataset, and try writing the result table into `sqlite` as shown above. Does information get lost?  Do you find it's easier to do operations in the database form, or the table form?  Why or why not?

While we have reached the end, it's worth noting this has only scratched the surface of data management - larger-scale mangement of workflows using tools like Apache Spark or Amazon Web Services to solve compute problems that could never be done at the single-computer levels.  So while that is out of scope for this work, hopefully some of the concepts developed here will help build your intuition for how to expand how you manage your data to whatever workflow you need.